In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import os

In [9]:
def load_data(ps):
    # test data
    test_df = pd.read_csv('data/'+str(ps)+'_test_exp.csv', header=None)
    test_df.rename(columns={2: 'condition', 3: 'completion'}, inplace=True)
    print 'The avg completion rate in treatment {}'.format(test_df[test_df['condition'] == 1]['completion'].mean())
    print 'The avg completion rate in control {}'.format(test_df[test_df['condition'] == 0]['completion'].mean())
    return test_df

### Post-model analysis

In [3]:
def post_analysis(file_name, test_df):
    test_df = load_data(ps)
    res_df = pd.read_csv(file_name, header=None)
    res_df = res_df.rename(columns={0: 'f', 1: 'cf'})
    concated_test_df = pd.concat([test_df, res_df], axis=1)
    concated_test_df['treatment_effect'] = np.where(concated_test_df['condition']==1, concated_test_df['f']-concated_test_df['cf'], \
                                           concated_test_df['cf']-concated_test_df['f'])

    # recommended condition
    concated_test_df['recommended_condition'] = np.where(concated_test_df['treatment_effect']>0, 1, 0)
    return concated_test_df

In [4]:
def calculate_completion(file_name, ps):
    concated_df = post_analysis(file_name, ps)
    concated_df[['condition', 'recommended_condition', 'completion']].head()

    matched_df = concated_df[((concated_df['condition'] == 1) & (concated_df['recommended_condition'] == 1)) | ((concated_df['condition'] == 0) & (concated_df['recommended_condition'] == 0))]
    unmatched_df = concated_df[((concated_df['condition'] == 1) & (concated_df['recommended_condition'] == 0)) | ((concated_df['condition'] == 0) & (concated_df['recommended_condition'] == 1))]

    print stats.ttest_ind(matched_df['completion'], unmatched_df['completion'])
    print matched_df['completion'].sum()*1.0/len(matched_df)
    print len(matched_df)
    print unmatched_df['completion'].sum()*1.0/len(unmatched_df)
    print len(unmatched_df)

In [11]:
ps = 263115
folder_path = 'results/archived/263115/'
test_df = load_data(ps)
for root, dirs, files in os.walk(folder_path):
    for name in dirs:
        if 'results' in name:
            print 'folder name: {}'.format(name)
            file_name = folder_path + name + '/y_pred.test_1.csv'
            try:
                calculate_completion(file_name, test_df)
            except:
                print 'Hello'
            print '*'*10

The avg completion rate in treatment 0.886075949367
The avg completion rate in control 0.885714285714
folder name: results_20180220_143415-795854
The avg completion rate in treatment 0.886075949367
The avg completion rate in control 0.885714285714
Ttest_indResult(statistic=0.976475071026026, pvalue=0.3304332985186277)
0.9102564102564102
78
0.8591549295774648
71
**********
folder name: results_20180220_143355-239259
The avg completion rate in treatment 0.886075949367
The avg completion rate in control 0.885714285714
Ttest_indResult(statistic=0.6814422837509144, pvalue=0.49666382167349143)
0.9041095890410958
73
0.868421052631579
76
**********
folder name: results_20180220_144122-530967
The avg completion rate in treatment 0.886075949367
The avg completion rate in control 0.885714285714
Ttest_indResult(statistic=0.6388871064024456, pvalue=0.523890595969781)
0.9012345679012346
81
0.8676470588235294
68
**********
folder name: results_20180220_143456-350306
The avg completion rate in treatme

In [ ]:
concated_df[['condition', 'recommended_condition', 'completion']].head()

In [ ]:
def calculate_treatment_effects(file_name):
    res_df = pd.read_csv(file_name, header=None)
    res_df = res_df.rename(columns={0: 'f', 1: 'cf'})
    concated_test_df = pd.concat([test_df, res_df], axis=1)
    concated_test_df['treatment_effect'] = np.where(concated_test_df[2]==1, concated_test_df['f']-concated_test_df['cf'], \
                                           concated_test_df['cf']-concated_test_df['f'])

    print 'avg treatment effect is {}'.format(np.mean(concated_test_df['treatment_effect']))
    a = concated_test_df['treatment_effect']
    print 'confidence interval is {}'.format(st.t.interval(0.95, len(a)-1, loc=np.mean(a), scale=st.sem(a)))